In [27]:
#####Uso de Hashtag
f =  open("./hashtags.txt", "r")
hashtagsList = []
hashtag = None
while True:
    # read line
    hashtag = f.readline()
    if (hashtag == "") :
        break
    hashtag = hashtag.rstrip('\n')
    hashtag = hashtag.replace('#', '')
    initDate = f.readline().rstrip('\n')
    lastDate = f.readline().rstrip('\n')
    hashtag = (hashtag, initDate, lastDate)
    hashtagsList.append(hashtag)
    # check if line is not empty
    
f.close()
for h in hashtagsList:
    print(h)


('aldjfolajsdfa', '01-02-2021', '16-04-2021')
('asfasdfasdfasdfa', '01-02-2021', '16-04-2021')
('PSOE', '01-02-2021', '09-04-2021')
('SiYoFueraDeMadrid', '16-04-2021', '16-04-2021')


In [57]:
def DayMonthYearToPetition(date):
    splited = ''
    if ('/' in date):
        splited = date.split('/')
    elif ('-' in date):
        splited = date.split('-')

    return (splited[2] + '-' + splited[1] + '-' + splited[0])

def PetitionToDayMonthYear(petition):
    splited = ''
    time = ''
    splited = petition.split('T')
    time = splited[1]
    splited = splited[0].split('-')
    return (splited[2] + '-' + splited[1] + '-' + splited[0] + 'T' + time)




# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# next_token: token de la page (inicialmente string vac'ia) que quieres buscar
# return tupla de n'umero de respuestas y next_token
def PetitionsLessEqual100(hashtag, start, end, number, bearer, next_token):

    startDate = DayMonthYearToPetition(start)
    endDate = DayMonthYearToPetition(end)
    total = (0, '')
    entradaizq = url + "query=%23" + hashtag + "&start_time=" +  startDate + "T00%3A00%3A00Z" + "&end_time=" + endDate + 'T11%3A59%3A59Z&max_results=' + str(number) + '&expansions=author_id&tweet.fields=created_at,lang,conversation_id&user.fields=created_at,entities'
    entradader = '"' + ' -H "Authorization: Bearer '+ bearer + '"'

    if (next_token != ''):
        entrada = entradaizq + '&next_token=' + next_token + entradader
    else:
        entrada = entradaizq + entradader

    #primera petici'on del hashtag
    json_obj = json.loads(check_output(entrada, shell=True).decode())

    # Comprobar que hay respuestas para dicho hashtag
    if ('meta' in json_obj) and (int(json_obj['meta']['result_count']) > 0):

        for tweet in json_obj['data']: 
            #imprimir id de usuario
            print("     author_id: " + tweet['author_id'])
            #imprimir id de tweet
            print("       tweet_id: " + tweet['id'])
            #imprimir hashtags
            print("       hashtags: " + hashtags)
            #imprimir fecha de creción
            print("       created_at: " + PetitionToDayMonthYear(tweet['created_at']) + "\n")
        if ('next_token' in json_obj['meta']):
            total = (int(json_obj['meta']['result_count']), json_obj['meta']['next_token'])
        else:
            total = (int(json_obj['meta']['result_count']), '')
        print("N'umero de peticiones enviadas: " + str(number))
        print("N'umero de peticiones recibidas: " + str(json_obj['meta']['result_count']))
        #df = pd.DataFrame(msgs)
        return total

    #si no tiene meta entonces es un error y hay que manejarlo
    elif ('meta' in  json_obj) and (int(json_obj['meta']['result_count']) == 0):
        print("No hay tweets para el Hashtag: " + str(hashtag))
    else:
        print("Error en la petición: ")
        # si el error es por cuestión de peticiones esperar para hacer la siguiente peticion
        if ('title' in json_obj):
            print("  Título del error: " + json_obj['title'])
            if ('Invalid Request' == json_obj['title']):
                print(entrada.split(' -H')[0])
        if ('status' in json_obj):
            print("  Código de error: " + json_obj['status'])
        if ('detail' in json_obj):
            print("  Descripción del error: " + json_obj['detail'])
    return total


# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# return devuelve el totl de tweets que se han obtenido
def TweetList(hashtag, start, end, number, bearer):
    total = 0
    resultado = (0, '')
    print("Inicio Hashtag: " + hashtag + "\\\\\\\\\\\\ \n")
    if (number < 100):
        resultado =  PetitionsLessEqual100(hashtag, start, end, number, bearer, None)
        time.sleep(TIMER)
    else:
        while ((number - total) > 0):
            #Caso base
            if ((number - total) >= 100):
                resultado = PetitionsLessEqual100(hashtag, start, end, 100, bearer, resultado[1])
            #Fin de caso
            elif ((number - total) > 0) and ((number - total) < 100):
                resultado = PetitionsLessEqual100(hashtag, start, end, (number - total), bearer, resultado[1])
            
            total += resultado[0]
            time.sleep(TIMER)
            # caso en el que la respuesta no encuentra tweets o no hay mas tweets
            if (resultado[0] == 0) or ((resultado[0] > 0) and resultado[1] == ''):
                break
    
    print ("Tweets totales: " + str(total))
    print("Fin Hashtag: " + str(hashtag) + "\\\\\\\\\\\\ \n\n")
    return resultado
    
    
    
            

In [58]:
#Versión para tweepy V2

#agregar que tipo de tweet es y a quien responden si es necesario
import tweepy
import csv
import pandas as pd
import json
from subprocess import check_output
import time

# CONSTANTES
PETITIONS_PER_TIME_LAPSE = 300
TIME_LAPSE = (15*60) + 300
TIMER = TIME_LAPSE/PETITIONS_PER_TIME_LAPSE

#variables
entrada = ""
next_token = None
entradaizq = '' 
entradader = ''
hashtags = ''
total = 0
startDate = ''
endDate = ''

url = 'curl "https://api.twitter.com/2/tweets/search/all?'


f = open("./accountV2.txt", "r")
Bearer = f.readline().rstrip('\n')
f.close()


for l in hashtagsList:
    TweetList(l[0], l[1], l[2], 121, Bearer)

    


Inicio Hashtag: aldjfolajsdfa\\\\\\ 

No hay tweets para el Hashtag: aldjfolajsdfa
Tweets totales: 0
Fin Hashtag: aldjfolajsdfa\\\\\\ 


Inicio Hashtag: asfasdfasdfasdfa\\\\\\ 

No hay tweets para el Hashtag: asfasdfasdfasdfa
Tweets totales: 0
Fin Hashtag: asfasdfasdfasdfa\\\\\\ 


Inicio Hashtag: PSOE\\\\\\ 

     author_id: 2755884282
       tweet_id: 1380490443402674179
       hashtags: 
       created_at: 09-04-2021T11:59:04.000Z

     author_id: 461213526
       tweet_id: 1380490187382390786
       hashtags: 
       created_at: 09-04-2021T11:58:03.000Z

     author_id: 460075199
       tweet_id: 1380490011435483143
       hashtags: 
       created_at: 09-04-2021T11:57:21.000Z

     author_id: 1247614386732249089
       tweet_id: 1380488677294542851
       hashtags: 
       created_at: 09-04-2021T11:52:03.000Z

     author_id: 2223225884
       tweet_id: 1380487765381550080
       hashtags: 
       created_at: 09-04-2021T11:48:26.000Z

     author_id: 956244054
       tweet_id: 138